In [7]:
import os
import pandas as pd
import datetime as dt
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from api_credentials import client_id, client_secret, redirect_uri
import json

# Load playlists from JSON file
with open('playlists.json', 'r') as file:
    playlists_dict = json.load(file)


# Initialize the Spotify client with client credentials for public data access
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

Read in the AU Daily 200 Spotify Chart  
Manually downloaded CSV from https://charts.spotify.com/charts/view/regional-au-daily/latest

In [8]:
# Load the CSV file
file_path = 'raw_data/regional-au-daily-2024-02-13.csv'
df = pd.read_csv(file_path)

# Extract the date from the file name
file_name = os.path.basename(file_path)  
date_str = file_name.split('-')[-3:]  # Splits the file name and takes the last 3 elements ['2024', '02', '05.csv']
date_str = '-'.join(date_str)  # Joins them back into '2024-02-05.csv'
date = date_str.split('.')[0]  # Removes the file extension, resulting in '2024-02-05'

# Convert the date string to a datetime object
date_datetime = pd.to_datetime(date)

# Format the datetime object to a string in 'DD-MM-YYYY' format
formatted_date = date_datetime.strftime('%d-%m-%Y')

# Add the formatted date as a new column to the DataFrame
df['date'] = formatted_date

Extract each tracks unique id for API call 

In [9]:
track_ids = df['uri'].str[14:]
df['track_id'] = df['uri'].str[14:]
df.drop(columns='uri', inplace=True)

Call Spotify API to retrieve `popularity_score` for each of the top 200 tracks

In [10]:
# Convert the Series to a list for processing
track_ids_list = track_ids.tolist()

# Spotify's `tracks` endpoint can take multiple track IDs at once,
# but to avoid potential rate limits or payload size issues,
# it's a good idea to process them in batches.
# Here, we define a batch size (e.g., 50 tracks per request)
batch_size = 50

# Initialize lists to store the retrieved data
artists = []
track_names = []
popularity_scores = []

for i in range(0, len(track_ids_list), batch_size):
    batch = track_ids_list[i:i+batch_size]
    tracks_info = sp.tracks(batch)
    for track in tracks_info['tracks']:
        # Some tracks might not be available, leading to `None` entries
        if track:
            # Extracting the primary artist name
            artist_name = track['artists'][0]['name'] if track['artists'] else 'N/A'
            artists.append(artist_name)
            
            # Extracting the track name
            track_names.append(track['name'])
            
            # Extracting the popularity score
            popularity_scores.append(track['popularity'])

# Now that we have all the data, put it into a DataFrame
popularity_df = pd.DataFrame({
    'artist_name': artists,
    'track_name': track_names,
    'popularity_score': popularity_scores
})

# Show the DataFrame 
popularity_df.head()


,artist_name,track_name,popularity_score
0,Noah Kahan,Stick Season,98
1,Benson Boone,Beautiful Things,96
2,Taylor Swift,Cruel Summer,99
3,Jack Harlow,Lovin On Me,99
4,Tate McRae,greedy,100


In [11]:
df['popularity_score'] = popularity_df['popularity_score']

In [12]:
# Reorder columns
df = df[['date', 'rank', 'artist_names', 'track_name', 'track_id', 'source', 'peak_rank', 'previous_rank', 'days_on_chart', 'streams', 'popularity_score']]

# Rename columns
df.rename(columns={
    'source': 'label',
    'streams': 'daily_streams',
}, inplace=True)

In [13]:
df.head()

,date,rank,artist_names,track_name,track_id,label,peak_rank,previous_rank,days_on_chart,daily_streams,popularity_score
0,13-02-2024,1,Noah Kahan,Stick Season,0mflMxspEfB0VbI1kyLiAv,Mercury Records/Republic Records,1,1,131,325133,98
1,13-02-2024,2,Benson Boone,Beautiful Things,6tNQ70jh4OwmPGpYy6R2o9,"Night Street Records, Inc./Warner Records Inc.",2,2,26,299697,96
2,13-02-2024,3,Taylor Swift,Cruel Summer,1BxfuPKGuaTgP7aM0Bbdwr,Taylor Swift,1,4,374,272030,99
3,13-02-2024,4,Jack Harlow,Lovin On Me,4xhsWYTOGcal8zt0J161CU,Generation Now/Atlantic,1,3,96,264218,99
4,13-02-2024,5,Tate McRae,greedy,3rUGC1vUpkDG9CZFHMur1t,RCA Records Label,1,6,152,218936,100


Retrieve additional data from Kworb.net
This will give us additional features/columns like:  
 - `count_at_peak`
 - `rank_movement`
 - `streams+`
 - `7day_streams`
 - `7day_streams_movement` 
 - `total_streams`

In [14]:
# Read the HTML table into a DataFrame
kworb_df = pd.read_html('https://kworb.net/spotify/country/au_daily.html')[0]

# Get the current date minus 2 days to account for data lag
current_date_minus_2 = dt.datetime.now().date() - dt.timedelta(days=2)

# Add the current date as a new column to the DataFrame
kworb_df['DATE'] = current_date_minus_2

In [15]:
kworb_df = kworb_df.rename(columns={
    'Pos' : 'rank',
    'P+': 'rank_change',
    'Artist and Title' : 'artist_title',
    'Days' : 'days_on_chart',
    'Pk' : 'peak_rank',
    '(x?)' : 'days_at_peak',
    'Streams' : 'daily_streams',
    'Streams+' : 'daily_streams_change',
    '7Day' : '7Day_streams',
    '7Day+' : '7Day_streams_change',
    'Total' : 'total_streams',
})

Need to change "=" to "0", remove "+", and handle "RE"

In [16]:
# Remove "+" and ensure "=" is replaced with "0"
kworb_df['rank_change'] = kworb_df['rank_change'].str.replace('+', '').str.replace('=', '0', regex=False)

# Attempt to convert the column to numeric without coercing errors to identify potential non-numeric values
try:
    kworb_df['rank_change'] = pd.to_numeric(kworb_df['rank_change'])
    print("Conversion successful. No NaN values detected.")
except ValueError as e:
    print("Conversion failed due to non-numeric values:", e)

# Optionally, inspect the column for any remaining non-numeric values or anomalies
# This step is just for verification and won't change the DataFrame
non_numeric = kworb_df[pd.to_numeric(kworb_df['rank_change'], errors='coerce').isna()]
if not non_numeric.empty:
    print("Non-numeric values found:\n", non_numeric)
else:
    print("No non-numeric values found.")

Conversion failed due to non-numeric values: Unable to parse string "RE" at position 153
Non-numeric values found:
      rank rank_change                                       artist_title  \
153   154          RE                          Taylor Swift - The Archer   
161   162          RE         Taylor Swift - Fearless (Taylor’s Version)   
163   164          RE                                 Lil Tecca - 500lbs   
167   168          RE                     Taylor Swift - illicit affairs   
174   175          RE                       Taylor Swift - Midnight Rain   
184   185          RE  Taylor Swift - I Knew You Were Trouble (Taylor...   
185   186          RE      Taylor Swift - Blank Space (Taylor's Version)   
197   198          RE                               Taylor Swift - the 1   
199   200          RE                 PARTYNEXTDOOR - Break from Toronto   

     days_on_chart  peak_rank days_at_peak  daily_streams  \
153             57          7          NaN          54861   
1

/var/folders/px/x65fp4pn31s5cv4gsbh_3cl80000gn/T/ipykernel_34125/641918597.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  kworb_df['rank_change'] = kworb_df['rank_change'].str.replace('+', '').str.replace('=', '0', regex=False)


In [17]:
# Extract numeric values from 'COUNT_AT_PEAK'
kworb_df['days_at_peak'] = kworb_df['days_at_peak'].str.extract('(\d+)', expand=False)

# Convert to numeric, allowing NaNs to remain
kworb_df['days_at_peak'] = pd.to_numeric(kworb_df['days_at_peak'], errors='coerce')

In [18]:
columns_to_keep = kworb_df[['artist_title', 'rank_change', 'days_at_peak', 'daily_streams_change', '7Day_streams', '7Day_streams_change', 'total_streams']]
columns_to_keep.head(3)

,artist_title,rank_change,days_at_peak,daily_streams_change,7Day_streams,7Day_streams_change,total_streams
0,Noah Kahan - Stick Season,0,18.0,23471.0,2275435,-21007,23632848
1,Benson Boone - Beautiful Things,0,6.0,17791.0,1967281,-41,5272820
2,Taylor Swift - Cruel Summer,1,1.0,33729.0,1752636,41332,53408647


In [19]:
print(f"Spotify Daily 200 columns:", df.columns.tolist())
print(f"Kworb Daily 200 columns:", kworb_df.columns.tolist())

Spotify Daily 200 columns: ['date', 'rank', 'artist_names', 'track_name', 'track_id', 'label', 'peak_rank', 'previous_rank', 'days_on_chart', 'daily_streams', 'popularity_score']
Kworb Daily 200 columns: ['rank', 'rank_change', 'artist_title', 'days_on_chart', 'peak_rank', 'days_at_peak', 'daily_streams', 'daily_streams_change', '7Day_streams', '7Day_streams_change', 'total_streams', 'DATE']


In [20]:
df['rank_change'] = kworb_df['rank_change']
df['days_at_peak'] = kworb_df['days_at_peak']
df['daily_streams_change'] = kworb_df['daily_streams_change']
df['7Day_streams'] = kworb_df['7Day_streams']
df['7Day_streams_change'] = kworb_df['7Day_streams_change']
df['total_streams'] = kworb_df['total_streams']


In [21]:
columns_order =  [
    'date',
    'artist_names',
    'track_name',
    'rank',
    'previous_rank',
    'rank_change',
    'peak_rank',
    'days_at_peak',
    'days_on_chart',
    'popularity_score',
    'daily_streams',
    'daily_streams_change',
    '7Day_streams',
    '7Day_streams_change',
    'total_streams',
    'label',
    'track_id',
]

df = df[columns_order]

In [22]:
df.head()

,date,artist_names,track_name,rank,previous_rank,rank_change,peak_rank,days_at_peak,days_on_chart,popularity_score,daily_streams,daily_streams_change,7Day_streams,7Day_streams_change,total_streams,label,track_id
0,13-02-2024,Noah Kahan,Stick Season,1,1,0,1,18.0,131,98,325133,23471.0,2275435,-21007,23632848,Mercury Records/Republic Records,0mflMxspEfB0VbI1kyLiAv
1,13-02-2024,Benson Boone,Beautiful Things,2,2,0,2,6.0,26,96,299697,17791.0,1967281,-41,5272820,"Night Street Records, Inc./Warner Records Inc.",6tNQ70jh4OwmPGpYy6R2o9
2,13-02-2024,Taylor Swift,Cruel Summer,3,4,1,1,1.0,374,99,272030,33729.0,1752636,41332,53408647,Taylor Swift,1BxfuPKGuaTgP7aM0Bbdwr
3,13-02-2024,Jack Harlow,Lovin On Me,4,3,-1,1,72.0,96,99,264218,10105.0,1901247,-21264,30404668,Generation Now/Atlantic,4xhsWYTOGcal8zt0J161CU
4,13-02-2024,Tate McRae,greedy,5,6,1,1,9.0,152,100,218936,13487.0,1575684,-20120,34544008,RCA Records Label,3rUGC1vUpkDG9CZFHMur1t


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   date                  200 non-null    object 
 1   artist_names          200 non-null    object 
 2   track_name            200 non-null    object 
 3   rank                  200 non-null    int64  
 4   previous_rank         200 non-null    int64  
 5   rank_change           200 non-null    object 
 6   peak_rank             200 non-null    int64  
 7   days_at_peak          42 non-null     float64
 8   days_on_chart         200 non-null    int64  
 9   popularity_score      200 non-null    int64  
 10  daily_streams         200 non-null    int64  
 11  daily_streams_change  191 non-null    float64
 12  7Day_streams          200 non-null    int64  
 13  7Day_streams_change   200 non-null    int64  
 14  total_streams         200 non-null    int64  
 15  label                 2

In [24]:
yesterday_df = pd.read_csv('combined_data/combined.csv')
yesterday_df.tail()

,date,artist_names,track_name,rank,previous_rank,rank_change,peak_rank,days_at_peak,days_on_chart,popularity_score,daily_streams,daily_streams_change,7Day_streams,7Day_streams_change,total_streams,label,track_id
1595,12-02-2024,P!nk,Raise Your Glass,196,185,RE,92,NaN,8,82,48279,NaN,193776,-2119,20132045,LaFace Records,1gv4xPanImH17bKZ9rOveR
1596,12-02-2024,Carly Rae Jepsen,Call Me Maybe,197,162,RE,113,NaN,288,85,47780,NaN,94684,45542,1439196,Silent Records IGA,20I6sIOMTCkB6w7ryavxtO
1597,12-02-2024,Hozier,Work Song,198,-1,RE,110,NaN,30,79,47556,NaN,141647,45484,34958688,Columbia,35PKfoynRpVFoAUE3D5Kc6
1598,12-02-2024,"JAY-Z, Kanye West",Ni**as In Paris,199,-1,-26,75,NaN,741,87,47233,-4860.0,372214,-5151,63601805,Roc Nation/RocAFella/IDJ,1auxYwYrFRqZP7t3s7w4um
1599,12-02-2024,Bryan Adams,Summer Of '69,200,173,RE,47,NaN,1674,85,47198,NaN,294622,-5312,17413380,A&M,0GONea6G2XdnHWjNZd6zt3


In [25]:
combined_df = pd.concat([yesterday_df, df], ignore_index=True)

In [26]:
print(len(combined_df))

1800


In [27]:
combined_df.to_csv('combined_data/combined.csv', index=False)